# Laboratory of Computational Physics mod. B at Phisics of Data, University of Padova

__Authors:__

- Valeria Fioroni (University of Padova)
- Matteo Guida (University of Padova)
- Philipp Zehetner (University of Padova, Ludwig Maximilian University of Munich)

__Supervised by:__

- Professor Marco Zanetti (University of Padova, CERN)
- Professor Stefano Lacaprara (University of Padova, BELLE2)

## 1. The Signal

At the SuperKEKB paricle accelerator electrons and positrons collide with a center-of-momentum energy clos to the $ \Upsilon (4S) $ resonance. The cross section for the event $ e^+ e^- \rightarrow \Upsilon (4S) $ is 

$$ 
\sigma_{\Upsilon (4S)} = 1.1 \, nb
$$

The $ \Upsilon (4S) $ decays in one of two (almost) equally probable products: 
- $ \Upsilon (4S) \rightarrow B^+ B^- $
- $ \Upsilon (4S) \rightarrow B^0 \bar{B^0} $

For this analysis we are interested in the decay channel
$$
B^0 \rightarrow \eta' \left( \eta \left( \gamma \gamma \right) \pi^+ \pi^- \right) K^0_S \left( \pi^+ \pi^- \right)
$$

The branching fractions are as follows: 

- $ \mathcal{BF} \left( B^0 \rightarrow \eta' K^0 \right) = 6.6 * 10^{-5} $ 
- $ \mathcal{BF} \left( \eta' \rightarrow \eta \pi^+ \pi^- \right) = 0.43 $
- $ \mathcal{BF} \left( \eta \rightarrow \gamma \gamma \right) = 0.40 $
- $ P \left( K^0_S | K^0 \right) = 0.5 $
- $ \mathcal{BF} \left( K^0_S \rightarrow \pi^+ \pi^- \right) = 0.7 $

This leads to a complete branching ratio of 
$ \mathcal{BF} = 3.97 * 10^{-6} $ 

## 2. Our Data

Our Data consists of labelled data, that we used to train and test a _Deep Neural Network_ (DNN) and a _Boosted Decision Tree_ as well as unlabelled data (Data Challenge) in which we are searching for our signal. 

###  Labelled Data

The labelled data consists of various types of background, all corresponding to an integrated luminosity of 
$ \mathcal{L} = 800 \, fb^{-1} $
and signal corresponding to __NSIGNAL__ reconstructed signal events with an efficiency of 
$ \varepsilon = 0.291 $

- $ e^+ e^- \rightarrow u \bar{u} $
- $ e^+ e^- \rightarrow d \bar{d} $
- $ e^+ e^- \rightarrow s \bar{s} $
- $ e^+ e^- \rightarrow c \bar{c} $
- $ e^+ e^- \rightarrow \tau \bar{\tau} $
- $ e^+ e^- \rightarrow B^+ B^- $
- $ e^+ e^- \rightarrow B^0 \bar{B^0} $

We summarize the first five types of backgrounds as _Continuum Background_ and the last two types as _Peaking Background_ and thus build a multi classifier with three types of events. 

Using the complete branching fraction 
$ \mathcal{BF} $
and the cross section 
$ \sigma_{\Upsilon (4S)} $ 
we can calculate to expect to have $N$ events with 
\begin{equation}
N = \mathcal{L} \times \sigma_{\Upsilon(4S)} \times \mathcal{BF} 
= 800 \, fb^{-1} \times 1.1 \, nb \times 3.97 \cdot 10^{-6}
= 3494
\end{equation}
of which $ \varepsilon \times N = 1017 $ should be found by the reconstruction

###  Data Challenge

The Data Challenge is a simulation corresponding to a total luminosity of 
$ \mathcal{L} = 1 \, ab^{-1} $
We can calculate in the same as above, that we should expect a total number of $ N = 4367 $ signal events of which $1271$ are reconstructed. 

In [16]:
import os, sys, pprint
import math
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import joblib
import warnings
warnings.filterwarnings("ignore")
from root_pandas import read_root
from random import seed
from random import randint
n_seed=1234
seed(n_seed)

Local_Philipp = True
Local_Valeria = False

In [9]:
Training = ['B0_Pcms', 'B0_etap_Pcms', 'B0_etap_eta_Pcms', 'B0_etap_eta_gamma0_Pcms', 'B0_etap_eta_gamma1_Pcms', 'B0_etap_pi0_Pcms', 'B0_etap_pi1_Pcms', 'B0_K_S0_Pcms', 'B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM', 'B0_deltae', 'B0_mbc', 'B0_TrCSMVA', 'B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho', 'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho', 'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y', 'B0_etap_eta_ErrY', 'B0_etap_eta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho', 'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z', 'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance', 'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr', 'B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc', 'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc', 'B0_VtxPvalue', 'B0_etap_VtxPvalue', 'B0_etap_eta_VtxPvalue', 'B0_etap_pi0_VtxPvalue', 'B0_etap_pi1_VtxPvalue', 'B0_K_S0_VtxPvalue']
Important = Training + ['B0_isSignal', 'evt_no']

if Local_Philipp : 
    path = '/home/philipp/Desktop/Project/DATA/'
    SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
    CFiles = glob.glob(os.path.join(path, 'Continuous/*.root'))
    PFiles = glob.glob(os.path.join(path, 'Peaking/*.root'))
    DCFiles = glob.glob(os.path.join(path, 'DC/*.root'))
    
if Local_Valeria : 
    path = '/home/utente/Scrivania/Progetto LCP-B/DataBelle2_all/'
    SFiles = glob.glob(os.path.join(path, 'Signal/*.root'))
    CFiles = glob.glob(os.path.join(path, 'Background/Continuum/*.root'))
    PFiles = glob.glob(os.path.join(path, 'Background/Peaking/*.root'))
    DCFiles = glob.glob(os.path.join(path, 'DataChallenge/*.root'))
    
Signal = pd.concat((read_root(f, 'B0', columns=Important) for f in SFiles))
Signal = Signal[Signal['B0_isSignal']==1].reset_index(drop=True)
Continuum_bkg = pd.concat((read_root(f, 'B0', columns=Important) for f in CFiles))
Peaking_bkg = pd.concat((read_root(f, 'B0', columns=Important) for f in PFiles))
DC = pd.concat((read_root(f, 'B0', columns=Training) for f in DCFiles)).reset_index(drop=True)

#Signal: In case of more than 1 candidate per event select the first one occuring
Mask_duplicated = Signal.duplicated(subset='evt_no', keep='first')
Mask_duplicated=np.logical_not(Mask_duplicated)
Signal=Signal[Mask_duplicated]
Signal.reset_index(drop=True, inplace=True)

print("Continuum: {} events".format(Continuum_bkg.shape[0]))
print("Peaking: {} events".format(Peaking_bkg.shape[0]))
print("Signal: {} events \n".format(Signal.shape[0]))
print("Assuming, that originally 1.6M signal events existed, this corresponds to a reconstruction efficiencey of {}%".format(100*Signal.shape[0]/1600000))

Continuum: 246329 events
Peaking: 2167 events
Signal: 218596 events
Assuming, that originally 1.6M signal events existed, this corresponds to a reconstruction efficiencey of 13.66225%


### Preprocessing our Data

- explain steps in preprocessing

In [12]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

n_classes = 3
Signal['Type'] = 2
Continuum_bkg['Type'] = 1
Peaking_bkg['Type'] = 0

# Splitting Data and Label into train, test and validation separatly for each class
X = Signal.drop('Type',axis=1)
Y = Signal['Type']
X_train_S, X_test_val_S, Y_train_S, Y_test_val_S = train_test_split(X, Y, train_size=0.5,random_state=randint(10**6,10**9))
X = X_test_val_S
Y = Y_test_val_S
X_validation_S, X_test_S, Y_validation_S, Y_test_S = train_test_split(X, Y, train_size=0.6,random_state=randint(10**6,10**9))
del(X_test_val_S)
del(Y_test_val_S)
X = Continuum_bkg.drop('Type',axis=1)
Y = Continuum_bkg['Type']
X_train_C, X_test_val_C, Y_train_C, Y_test_val_C = train_test_split(X, Y, train_size=0.5,random_state=randint(10**6,10**9))
X = X_test_val_C
Y = Y_test_val_C
X_validation_C, X_test_C, Y_validation_C, Y_test_C = train_test_split(X, Y, train_size=0.6,random_state=randint(10**6,10**9))
del(X_test_val_C)
del(Y_test_val_C)
X = Peaking_bkg.drop('Type',axis=1)
Y = Peaking_bkg['Type']
X_train_P, X_test_val_P, Y_train_P, Y_test_val_P = train_test_split(X, Y, train_size=0.5,random_state=randint(10**6,10**9))
X = X_test_val_P
Y = Y_test_val_P
X_validation_P, X_test_P, Y_validation_P, Y_test_P = train_test_split(X, Y, train_size=0.6,random_state=randint(10**6,10**9))
del(X_test_val_P)
del(Y_test_val_P)

# Concatenating the different classes, encoding the label as one-hot vectors and shuffling everything
X_Train=np.concatenate((X_train_S, X_train_C, X_train_P), axis=0)
X_Validation=np.concatenate((X_validation_S, X_validation_C, X_validation_P), axis=0)
X_Test=np.concatenate((X_test_S, X_test_C, X_test_P), axis=0)
Y_Train=np.concatenate((Y_train_S, Y_train_C, Y_train_P), axis=0)
Y_Validation=np.concatenate((Y_validation_S, Y_validation_C, Y_validation_P), axis=0)
Y_Test=np.concatenate((Y_test_S, Y_test_C, Y_test_P), axis=0)
Ỳ_Test_not_encoded=Y_Test #used for cut value analysis
Y_Train=to_categorical(Y_Train, num_classes=3)
Y_Validation=to_categorical(Y_Validation, num_classes=3)
Y_Test=to_categorical(Y_Test, num_classes=3)
permutation = np.random.permutation(X_Train.shape[0])
X_Train = X_Train[permutation]
Y_Train = Y_Train[permutation]
permutation = np.random.permutation(X_Validation.shape[0])
X_Validation = X_Validation[permutation]
Y_Validation = Y_Validation[permutation]
permutation = np.random.permutation(X_Test.shape[0])
X_Test = X_Test[permutation]
Y_Test = Y_Test[permutation]

Using TensorFlow backend.


Write something about the 5 sigma mass cut and why we aren't doing it
It's something we tried, so worth mentioning

Write something about scaling and why it is important

In [14]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
pcTrain = ss.fit_transform(X_Train)
pcTest = ss.transform(X_Test)
pcValidation = ss.transform(X_Validation)

Write something about PCA, and why we are not doing it

## 3. The Neural Network

explain what neural network we are using, how we have been training it

In [18]:
from keras.models import load_model

Model = load_model('Models/LastTry/Best_Model.h5')
Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_148 (Dropout)        (None, 83)                0         
_________________________________________________________________
dense_662 (Dense)            (None, 50)                4200      
_________________________________________________________________
dense_663 (Dense)            (None, 100)               5100      
_________________________________________________________________
dense_664 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_665 (Dense)            (None, 50)                5050      
_________________________________________________________________
dense_666 (Dense)            (None, 3)                 153       
Total params: 24,603
Trainable params: 24,603
Non-trainable params: 0
_________________________________________________________________


Talk about the performance of our model 

- Model.evaluate with Test sample
- ROCK Curve
- Confusion Matrix

## 4. XGBOOST

- Same story here as in the last chapter
- Advantages/Disadvantages compared to the DNN

## 5. Deciding on the Cut Value of our Network

- all necessary steps and explanations for the cut value

## 6. The Data Challenge

Here we con showcase the perfect performance of our network

## 7. Conclusion or additional comments (optional)

If you think somthing is missing feel free to add it. 
With 6 topics we can give 2 (non consecutive) topics for everybody. 
Having a little break between your two topics maybe helps a bit if something didn't go as expected. 